<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS3_withAtten/blob/main/ASS3_WithAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and login

In [ ]:
!pip install pytorch_lightning
!pip install wandb


In [ ]:
import wandb
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
import torch
import torch.nn.functional as F
torch.cuda.empty_cache()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch.utils.data as data
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
warnings.filterwarnings("ignore")
#function to create 3*3 grid of heatMap
import matplotlib.font_manager as fm


In [ ]:
wandb.login(key="7ff4b917f4a107c44e50219b5fd09ece84a34600")
# wandb.init( project="CS6910_Assignment3")
# !wandb login 494428cc53b5c21da594f4fc75035d136c63a93c


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Data Preprocessing

In [ ]:
def read_file_0(filepath):
    def process_row(text_row):
        return list(text_row)

    def collect_characters(reader_obj):
        all_chars = []
        for entry in reader_obj:
            all_chars += process_row(entry[0])
        return all_chars

    with open(filepath, mode='r') as file_handle:
        csv_reader = csv.reader(file_handle)
        return collect_characters(csv_reader)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'''Location of your CSV file (Extracted file)
Location of your CSV file if on kaggle than zip file is fine'''

trainFilepath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_train.csv"
valFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_valid.csv"
testFilePath="/kaggle/input/myinput/aksharantar_sampled/hin/hin_test.csv"

In [ ]:


chars = read_file_0(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

# Create the association between characters and their corresponding integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
def read_file_1(trainFilepath):
    def extract_chars(text):
        return list(text)

    def accumulate(reader_obj):
        buffer = []
        for entry in reader_obj:
            buffer += extract_chars(entry[1])
        return buffer

    with open(trainFilepath, 'r') as file_stream:
        csv_rows = csv.reader(file_stream)
        return accumulate(csv_rows)

In [ ]:
maxLenDev=0

chars = read_file_1(trainFilepath)
setChar=set(chars)
setChar.add('|')
setOfchar = list(setChar)

charToIndLang ={char: i+1 for i, char in enumerate(setOfchar)}

In [ ]:
with open(trainFilepath, 'r') as csv_file:
    reader = csv.reader(csv_file)
    max_length = 0

    def update_max(current_max, candidate):
        return candidate if candidate > current_max else current_max

    for record in reader:
        length = len(record[0])
        max_length = update_max(max_length, length)

    maxLenEng = max_length

In [ ]:
with open(trainFilepath, 'r') as csv_stream:
    reader_obj = csv.reader(csv_stream)
    longest = 0

    def get_larger(a, b):
        return b if b > a else a

    for line in reader_obj:
        current_length = len(line[1])
        longest = get_larger(longest, current_length)

    maxLenDev = longest